# Large Language Models for Education

In this notebook, we demonstrate how to use large language models (LLMs) for use cases in education.  LLMs can be used for tasks such as summarization, question-answering, or the generation of question & answer pairs.

The first part of this notebook sets up a SageMaker endpoint, using a **FLAN T5** model, on a single `ml.p3.2xlarge` instance.  Next we show how the endpoint can be queried, and some subroutines are added to demonstrate prompts for summarization, question-answering and the generation of question & answer pairs.

The final section is split into four demos on querying the Wikipedia article on quantum computing, an ebook text from [Project Gutenberg](https://www.gutenberg.org/), a scientific pdf article from arxiv.org, and the Australian Budget 2023-24 Medicare Overview.

## 1.Setting up the SageMaker Endpoint

### 1.1 Install Python Dependencies

In [3]:
%%capture
!pip install --upgrade pip
!pip install -U sagemaker
!pip install -U langchain
!pip install -U PyPDF2

### 1.2 Deploying a SageMaker Endpoint

In [4]:
import sagemaker, boto3, json, logging
from sagemaker import image_uris, instance_types, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.session import Session
from sagemaker.utils import name_from_base
from IPython.display import display, HTML, IFrame

In [5]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [6]:
logger.info(f'Using sagemaker=={sagemaker.__version__}')
logger.info(f'Using boto3=={boto3.__version__}')

Using sagemaker==2.168.0
Using boto3==1.26.154


In [7]:
# Create the folder where the model weights will be stored
!mkdir -p download_dir
!mkdir -p source_documents_dir

In [8]:
def get_sagemaker_session(local_download_dir) -> sagemaker.Session:
    """Return the SageMaker session."""

    sagemaker_client = boto3.client(
        service_name="sagemaker", region_name=boto3.Session().region_name
    )

    session_settings = sagemaker.session_settings.SessionSettings(
        local_download_dir=local_download_dir
    )

    # the unit test will ensure you do not commit this change
    session = sagemaker.session.Session(
        sagemaker_client=sagemaker_client, settings=session_settings
    )

    return session

In [9]:
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

model_id, model_version = "huggingface-text2text-flan-t5-xl", "*"
_model_env_variable_map = {
    "huggingface-text2text-flan-t5-xl": {"MMS_DEFAULT_WORKERS_PER_MODEL": "1"},
}

endpoint_name = name_from_base(f"jumpstart-example-{model_id}")
instance_type = 'ml.p3.2xlarge'
logger.info(f'Using role {aws_role} in region {aws_region}')

Using role arn:aws:iam::278313627171:role/service-role/AmazonSageMaker-ExecutionRole-20221130T152651 in region us-east-1


In [10]:
# Retrieve the inference docker container uri. This is the base HuggingFace container image for the default model above.
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=instance_type,
)

# Retrieve the inference script uri. This includes all dependencies and scripts for model loading, inference handling etc.
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

# Retrieve the model uri.
model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="inference"
)

# Create the SageMaker model instance
if model_id in _model_env_variable_map:
    # For those large models, we already repack the inference script and model
    # artifacts for you, so the `source_dir` argument to Model is not required.
    model = Model(
        image_uri=deploy_image_uri,
        model_data=model_uri,
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        env=_model_env_variable_map[model_id],
    )
else:
    model = Model(
        image_uri=deploy_image_uri,
        source_dir=deploy_source_uri,
        model_data=model_uri,
        entry_point="inference.py",  # entry point file in source_dir and present in deploy_source_uri
        role=aws_role,
        predictor_cls=Predictor,
        name=endpoint_name,
        sagemaker_session=get_sagemaker_session("download_dir"),
    )

In [11]:
%%time
# deploy the Model. Note that we need to pass Predictor class when we deploy model through Model class,
# for being able to run inference through the sagemaker API.
print(f'Deploying endpoint {endpoint_name} on 1 x {instance_type} (this will take approximately 6-8 minutes)')
try:
    model_predictor = model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        predictor_cls=Predictor,
        endpoint_name=endpoint_name,
    )
except Exception as e:
    print(f'Error: {e}')
    print('Two common reasons for this error')
    print('1. You are in a AWS region that does not have the ml.p3.2xlarge instance type')
    print('2. You have exceeded the service quota of this AWS account')

Creating model with name: jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174
CreateModel request: {
    "ModelName": "jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174",
    "ExecutionRoleArn": "arn:aws:iam::278313627171:role/service-role/AmazonSageMaker-ExecutionRole-20221130T152651",
    "PrimaryContainer": {
        "Image": "763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04",
        "Environment": {
            "MMS_DEFAULT_WORKERS_PER_MODEL": "1"
        },
        "ModelDataUrl": "s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.1.2/infer-prepack-huggingface-text2text-flan-t5-xl.tar.gz"
    },
    "Tags": [
        {
            "Key": "aws-jumpstart-inference-model-uri",
            "Value": "s3://jumpstart-cache-prod-us-east-1/huggingface-infer/prepack/v1.1.2/infer-prepack-huggingface-text2text-flan-t5-xl.tar.gz"
        }
    ]
}


Deploying endpoint jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174 on 1 x ml.p3.2xlarge (this will take approximately 6-8 minutes)


Creating endpoint-config with name jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174
Creating endpoint with name jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174


-----------!CPU times: user 142 ms, sys: 28.5 ms, total: 170 ms
Wall time: 6min 3s


In [12]:
print(f'Successfully deployed endpoint {endpoint_name} on 1 x {instance_type}')

Successfully deployed endpoint jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174 on 1 x ml.p3.2xlarge


## 2. Subroutines for text extraction and for querying the endpoint

In [13]:
import pandas as pd
import PyPDF2
import re
import requests

from bs4 import BeautifulSoup

pd.set_option('max_colwidth', 80)  # Set max column width for displaying Pandas Dataframes
QNA_OUTPUT_STYLE = 'HTML'

### 2.1 Extract text from a PDF file

Using the PyPDF library, we can extract text from a PDF file.

In [14]:
def extract_pages(pdf_file, max_pages=100):
    pages = []
    with open(pdf_file, 'rb') as f:
        for i, page in enumerate(PyPDF2.PdfReader(f).pages):
            if i == max_pages:
                break
            pages.append(page.extract_text())
    return pages

### 2.2 Download a text-based ebook from a URL

In [15]:
def download_url_text(url):
    r = requests.get(url)
    if r.status_code == 200:
        return r.content.decode('utf-8')
    else:
        print(f'Failed to download {url}. Status code = {r.status_code}')
        return None

### 2.3 Extract paragraphs from a web page

Using the Beautiful Soup library, we can parse the HTML content of a web page.
Here, the text from the `<p>` tags in the `<body>` section the page are extracted.

In [16]:
def extract_paragraphs_from_html(text):
    html = BeautifulSoup(text, 'html.parser')
    return [ p.text for p in html.body.select('p') ]

### 2.4 Generate texts, or query an endpoint with prompts

In [17]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
lightred, lightgreen, lightyellow, lightblue = '\033[91m', '\033[92m', '\033[93m', '\033[94m'
lightmagenta, lightcyan, reset = '\033[95m', '\033[96m', '\33[39m'

def query_endpoint_with_json_payload(encoded_json):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=encoded_json)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    generated_text = model_predictions['generated_texts']
    return generated_text

def generate_text_from_prompt(prompt, max_length=300, max_time=50, temperature=0.5,
                              top_k=None, top_p=None, do_sample=True, seed=None):
    payload = {
        "text_inputs": prompt,
        "max_length": max_length,
        "max_time": max_time,
        "temperature": temperature,
        "do_sample": do_sample
    }
    if top_k is not None:
        payload['top_k'] = top_k
    if top_p is not None:
        payload['top_p'] = top_p
    if seed is not None:
        payload['seed'] = seed

    query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
    return parse_response_multiple_texts(query_response)[0]

To further customize the outputs of the large language model, the following parameters are available:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_time:** The maximum amount of time you allow the computation to run for in seconds. Generation will still finish the current pass after allocated time has been passed. This setting can help to generate a response prior to endpoint invocation response time out errors.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.


In [18]:
def summarize(text, seed=None):
    return generate_text_from_prompt(
        f"""Summarize the following text in 100 words:\n\n{text}\n\nSummary:""",
        temperature=0.2,  # Low temperature for summarization
        seed=seed
    )

def ask(context, question, seed=None):
    return generate_text_from_prompt(
        f"""CONTEXT:\n{context}\n{question}""",
        temperature=0.01,  # Lowest temperature for accuracy
        max_length=150,    # Keep answers from being too verbose
        seed=seed
    )

def extract_question(text, seed=None):
    return generate_text_from_prompt(
        f"""EXTRACT QUESTIONS\nContext:\n{text}\nQuestion:""",
        temperature=1.0,  # Maximum temperature for creativity
        seed=seed
    )

def create_qna_pairs(text, n, output_style='HTML', seed=None):
    questions = []
    answers = []

    for i in range(n):
        qn = extract_question(text, seed) if i == 0 else extract_question(text)
        questions.append(qn)
        answers.append(ask(text, qn))
        if output_style == 'HTML':
            output = \
            f"""<b>{i+1}</b>. <b><font color=#FF7F50>Question</font></b>: {questions[i]}
            <b><font color=#FA8072>Answer</font></b>: {answers[i]}"""
            display(HTML(output))
        elif output_style == 'text':
            print(f"""{i+1}. {lightblue}{bold}Question{unbold}{reset}: {questions[i]} {lightcyan}{bold}Answer{unbold}{reset}: {answers[i]}""")
    if output_style == 'table':
        return pd.DataFrame({
            'Question': questions,
            'Answer': answers
        }).drop_duplicates()

## 3. LLM Demos for Education

In this notebook, we use the following texts to demonstrate summarization tasks and the generation of question & answer pairs.

1. Quantum Computing from Wikipedia: https://en.wikipedia.org/wiki/Quantum_computing
<!-- 1. Quantum Computing and Quantum Information (by Nielsen & Chuang): https://michaelnielsen.org/qcqi/QINFO-book-nielsen-and-chuang-toc-and-chapter1-nov00.pdf (this is a sample chapter from [this website](https://michaelnielsen.org/qcqi/)) -->
2. Winnie the Pooh (by Alan Alexander Milne): https://www.gutenberg.org/ebooks/67098.txt.utf-8
3. Attention is all you need (by Vaswani et al): https://arxiv.org/pdf/1706.03762.pdf
4. Australian Budget 2023-24 Overview: https://budget.gov.au/content/overview/download/budget_overview-20230511.pdf

Note that for this notebook, we are using the Flan T5 XL model for simplicity and ease of deployment--additional fine tuning or using improved models would be required to get better results.

In [19]:
# Download pdfs and texts with the `curl` command. Flags used here are `-L` (allow redirects),
# `-s` (for silent mode) and `-o` (to specify the output file name).

# Attention is all you need (by Vaswani et al)
!curl -Ls https://arxiv.org/pdf/1706.03762.pdf -o source_documents_dir/attention.pdf
# Australian Budget 2023-24 Overview
!curl -Ls https://budget.gov.au/content/overview/download/budget_overview-20230511.pdf -o source_documents_dir/aus_budget_overview-2023-24.pdf

### 3.1 Wikipedia Page on Quantum Computing

In this example, a Wikipedia page on Quantum Computing is used for context. The LLM is used for keyword generation, a point by point summary, and a set of question and answer pairs. You may also wish to replace the Wikipedia URL with a website, blog, or news article of your own preference.

In [20]:
NCHARS = 400     # We will show just the first and last 400 characters of each extracted text. Increase this number for more context.
NQUESTIONS = 10  # The number of Q&A pairs that we will generate.

In [21]:
wiki_paragraphs = extract_paragraphs_from_html(
    download_url_text('https://en.wikipedia.org/wiki/Quantum_computing')
)[1:11]  # We will skip the first 2 paragraphs
wiki_txt = '\n\n'.join(wiki_paragraphs)
# print(f'{txt1[:NCHARS]}...\n\n...{txt1[-NCHARS:]}')
IFrame('https://en.wikipedia.org/wiki/Quantum_computing', width=800, height=300)

#### Key word Generation

In [22]:
KEY_WORDS = generate_text_from_prompt(
    f'FIND KEY WORDS\n\nContext:\n{wiki_txt}\nKey Words:',
    seed=12345
)
key_word_list = KEY_WORDS.split(', ')
print(KEY_WORDS)

quantum, computing, superposition, qubit


#### Summary of key points

For each of each of paragraphs, let's create a short summary.

In [23]:
summary = []
for i, x in enumerate(wiki_paragraphs):
    summary.append(f'{i+1}. {summarize(x[:1500])}')

In [24]:
HTML(
    '<h4>Key Points</h4>' + 
    '\n'.join([ f'<li>{x}</li>' for x in summary ])
)

In [25]:
# The 10 points above can be used to create an even shorter summary.
summarize('\n'.join(summary))

'Quantum computing is the development of a computer that uses quantum physics to perform computations that are impossible for classical computers.'

#### Checking for correct answers

In this example, we generate a "correct answer" based on the text. One incorrect answer,
and one correct answer (paraphrased slightly differently from the official "correct answer")
from a student are generated. The LLM is used to check if the student's answer is correct.

In [26]:
prompt=f"""Context:{wiki_txt}
What is quantum computing?"""
answer = generate_text_from_prompt(prompt, temperature=0.01)
print(answer)

A quantum computer is a computer that exploits quantum mechanical phenomena.


In [27]:
prompt=f"""Context:{wiki_txt}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing is using computers with quantum dots
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

no


In [28]:
prompt=f"""Context:{wiki_txt}
Question: What is quantum computing?
Answer: {answer}
Student: Quantum computing involves using computers that make use of quantum mechanics
Is this answer correct?"""
print(generate_text_from_prompt(prompt, temperature=0.01))

yes


#### Generation of Question & Answer Pairs

In [29]:
create_qna_pairs(wiki_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE, seed=1234)

### 3.2 Winnie the Pooh (by Alan Alexander Milne)

In [30]:
winnie_the_pooh = download_url_text('https://www.gutenberg.org/ebooks/67098.txt.utf-8')

In [31]:
x = winnie_the_pooh.find('CHAPTER III')
pooh_txt = winnie_the_pooh[x:x+5000]  # Extract the first 5000 characters of chapter 3
print(pooh_txt)

CHAPTER III

                   IN WHICH POOH AND PIGLET GO HUNTING
                        AND NEARLY CATCH A WOOZLE


The Piglet lived in a very grand house in the middle of a beech-tree,
and the beech-tree was in the middle of the forest, and the Piglet lived
in the middle of the house. Next to his house was a piece of broken
board which had: "TRESPASSERS W" on it. When Christopher Robin asked the
Piglet what it meant, he said it was his grandfather's name, and had
been in the family for a long time, Christopher Robin said you
_couldn't_ be called Trespassers W, and Piglet said yes, you could,
because his grandfather was, and it was short for Trespassers Will,
which was short for Trespassers William. And his grandfather had had two
names in case he lost one--Trespassers after an uncle, and William after
Trespassers.

"I've got two names," said Christopher Robin carelessly.

"Well, there you are, that proves it," said Piglet.

One fine winter's day when Piglet was brushing away the s

In [32]:
ask(pooh_txt, "What is the storyline here?")

'The Piglet lived in a very grand house in the middle of a beech-tree, and the beech-tree was in the middle of the forest, and the Piglet lived in the middle of the house. Next to his house was a piece of broken board which had: "TRESPASSERS W" on it. When Christopher Robin asked the Piglet what it meant, he said it was his grandfather\'s name, and had been in the family for a long time. Christopher Robin said you _could_ be called Trespassers W, and Piglet said yes, you could, because his grandfather was, and it was short for Trespassers Will'

In [33]:
ask(pooh_txt, "Who is the main character?")

'Winnie-the-Pooh'

In [34]:
ask(pooh_txt, "What happens at the end?")

'Winnie-the-Pooh and Piglet nearly catch a Woozle.'

In [35]:
create_qna_pairs(pooh_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE, seed=12345)

### 3.3 Attention is all you need (by Vaswani et al)

In [36]:
attention = extract_pages('source_documents_dir/attention.pdf')

In [37]:
attention_txt = '\n\n'.join(attention[1:3] + attention[9:10])  # We will use pages 1, 2 (for the intro), and 9 (for the conclusion)
# print(f'{attention_txt[:NCHARS]}...\n\n\n...{attention_txt[-NCHARS:]}')
IFrame('source_documents_dir/attention.pdf', width=800, height=400)

#### Question Answering

In [38]:
ask(attention_txt, "What is the main gist of the paper?")

'We propose the Transformer, a model architecture eschewing recurrence and instead relying entirely on an attention mechanism to draw global dependencies between input and output. The Transformer allows for significantly more parallelization and can reach a new state of the art in translation quality after being trained for as little as twelve hours on eight P100 GPUs.'

In [39]:
ask(attention_txt, "What is the problem being solved?")

'The Transformer is the first sequence transduction model relying entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.'

In [40]:
ask(attention_txt, "What is the conclusion of the paper?")

'Transformer is the first sequence transduction model based entirely on attention, replacing the recurrent layers most commonly used in encoder-decoder architectures with multi-headed self-attention.'

In [41]:
chunk_size = len(attention_txt)//8
print(f'The text will be split up into chunks of {chunk_size} characters and summarized')

The text will be split up into chunks of 1202 characters and summarized


In [42]:
display(HTML('<h4>Key Points</h4>'))
summary = []
for i in range(8):
    x0 = i*chunk_size
    x1 = (i+1)*chunk_size
    line_summary = f'{i+1}. {summarize(attention_txt[x0:x1])}'
    display(HTML(line_summary))
    summary.append(line_summary)

In [43]:
create_qna_pairs(attention_txt, NQUESTIONS, output_style=QNA_OUTPUT_STYLE)

### 3.4 Australian Budget 2023-24 Overview (Medicare)

In this example, we look at the Australian Budget 2023-24 and we focus on the Medicare improvements.

In [44]:
# Extracting the pages from the Budget overview and work on the pages 24 to 27 (Medicare related)
aus_budget_overview = extract_pages('source_documents_dir/aus_budget_overview-2023-24.pdf')
txt_aus_budget_overview_medicare = '\n\n'.join(aus_budget_overview[24:27])  # We will use pages 24 to 27. Those pages cover the Medicare budget.
print(f'{txt_aus_budget_overview_medicare[:NCHARS]}...\n\n\n...{txt_aus_budget_overview_medicare[-NCHARS:]}')

Historic investment in Medicare 
Strengthening Medicare
Medicare is the foundation of Australia’s primary health care system. In this 
Budget, the Government is investing $5.7 billion over 5 years from 2022—23 to 
strengthen Medicare and make it cheaper and easier to see a doctor.
The Strengthening Medicare package includes the largest investment in bulk 
billing incentives ever. The Government is...


...llion over 4 years to establish the Primary Care and Midwifery 
Scholarships program, supporting registered nurses and midwives in 
post-graduate study to improve their skills 
• $31.6 million over 2 years for improved training arrangements for 
international medical students working rur al and remote locations.
26 Strengthening MedicareStronger foundations for a better future   |   Budget 2023–24


#### Question Answering

In [45]:
summarize(txt_aus_budget_overview_medicare)

'The Government is investing $5.7 billion over 5 years from 2022-23 to strengthen Medicare and make it cheaper and easier to see a doctor.'

In [46]:
create_qna_pairs(txt_aus_budget_overview_medicare, 5, output_style=QNA_OUTPUT_STYLE)

In [47]:
ask(txt_aus_budget_overview_medicare,"What is a Level B consultation?")

'telehealth general practice services which are between 6 and 20 minutes in length'

In [48]:
ask(txt_aus_budget_overview_medicare, "How much is the govermement investing?")

'$5.7 billion'

In [49]:
ask(txt_aus_budget_overview_medicare, "Is the governement helping the homeless people?")

'The Government will also invest in new services to help homeless people and culturally and linguistically diverse communities to access primary care.'

## 4. [Optional] LLM Demos for Education Part II

In this section, we deploy a Gradio app that takes a URL as input, and allows us to answer questions based on the content of the web page.

### 4.1 Gradio Demo App

In [50]:
%%capture
!pip install gradio

In [51]:
import gradio as gr

/opt/conda/lib/python3.7/site-packages/pkg_resources/__init__.py:126: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  PkgResourcesDeprecationWarning,


In [52]:
def url2context(url):
    paragraph_list = extract_paragraphs_from_html(
        download_url_text(url)
    )[1:11]  # We will skip the first paragraph, and take only 10 paragraphs
    return '\n\n'.join(paragraph_list)

In [53]:
def chatbot(prompt, temperature, max_length, url):
    if url == "":
        return generate_text_from_prompt(prompt, max_length, temperature)
    else:
        return ask(url2context(url), prompt)

def summary(url):
    context = url2context(url)
    key_words = generate_text_from_prompt(
        f'FIND KEY WORDS\n\nContext:\n{context}\nKey Words:'
    )
    return f"""{summarize(context)}\n\nKey words: {key_words}"""

with gr.Blocks() as demo:
    gr.Markdown("## Flan T5 Chatbot Demo")
    with gr.Row():
        with gr.Column():
            url = gr.Textbox(label="URL", placeholder="Enter URL here", lines=1, show_label=True,
                             value="https://mmrjournal.biomedcentral.com/articles/10.1186/s40779-022-00416-w"
                             # value="https://k12.libretexts.org/Bookshelves/Science_and_Technology/Biology/03%3A_Genetics/3.14%3A_Human_Genome"
                            )
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(
                label="Prompt", placeholder="Enter your prompt here", lines=3, show_label=True,
                value=f"How do mRNA vaccines work for pancreatic cancer treatment?")
            temperature = gr.Slider(label="Temperature", minimum=0.0, maximum=1.0, value=0.5)
            max_length = gr.Slider(label="Max Length", minimum=20, maximum=400, value=100)
        with gr.Column():
            output = gr.Textbox(label="Output", lines=10, show_label=True)
    with gr.Row():
        with gr.Column():
            submit_btn = gr.Button("Submit")
        with gr.Column():
            summary_btn = gr.Button("Summary")
    submit_btn.click(
        fn=chatbot,
        inputs=[prompt, temperature, max_length, url],
        outputs=output,
        api_name="chatbot",
        queue=False
    )
    summary_btn.click(
        fn=summary,
        inputs=[url],
        outputs=output,
        api_name="summary",
        queue=False
    )

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c6a7474012814cf276.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## 5. Cleanup

Delete the SageMaker model and endpoint

In [54]:
model_predictor.delete_model()
model_predictor.delete_endpoint()

Deleting model with name: jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174
Deleting endpoint configuration with name: jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174
Deleting endpoint with name: jumpstart-example-huggingface-text2text-2023-06-23-05-04-38-174


To completely shutdown SageMaker, go to File > Shut Down > Shutdown All